In [1]:
from transformers import AutoTokenizer
from PIL import Image
from vllm import LLM, SamplingParams

MODEL_NAME = "HwwwH/MiniCPM-V-2"
# Also available for previous models
# MODEL_NAME = "openbmb/MiniCPM-V-2_6"
# MODEL_NAME = "openbmb/MiniCPM-Llama3-V-2_5"
# MODEL_NAME = "HwwwH/MiniCPM-V-2"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
llm = LLM(
    model=MODEL_NAME,
    trust_remote_code=True,
    gpu_memory_utilization=0.8,
    dtype='float16',
    max_model_len=2048
)


/home/nandouser/minicpm-v/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-09-03 13:47:02,311	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


WARNING 09-03 13:47:11 config.py:1563] Casting torch.bfloat16 to torch.float16.
INFO 09-03 13:47:11 llm_engine.py:184] Initializing an LLM engine (v0.5.5) with config: model='HwwwH/MiniCPM-V-2', speculative_config=None, tokenizer='HwwwH/MiniCPM-V-2', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=2048, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=HwwwH/MiniCPM-V-2, use_v2_block_manager=False, enable_prefix_caching=False)
WARNING 09-03

/home/nandouser/minicpm-v/lib/python3.10/site-packages/xformers/ops/fmha/flash.py:211: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_fwd")
/home/nandouser/minicpm-v/lib/python3.10/site-packages/xformers/ops/fmha/flash.py:344: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_bwd")


INFO 09-03 13:47:15 model_runner.py:879] Starting to load model HwwwH/MiniCPM-V-2...
INFO 09-03 13:47:15 selector.py:217] Cannot use FlashAttention-2 backend for Volta and Turing GPUs.
INFO 09-03 13:47:15 selector.py:116] Using XFormers backend.
INFO 09-03 13:47:17 weight_utils.py:236] Using model weights format ['*.safetensors']


Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:27<00:27, 27.14s/it]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [01:39<00:00, 53.91s/it]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [01:39<00:00, 49.89s/it]



INFO 09-03 13:48:57 model_runner.py:890] Loading model weights took 6.4378 GB


/home/nandouser/minicpm-v/lib/python3.10/site-packages/transformers/models/auto/image_processing_auto.py:513: FutureWarning: The image_processor_class argument is deprecated and will be removed in v4.42. Please use `slow_image_processor_class`, or `fast_image_processor_class` instead
  warnings.warn(


WARNING 09-03 13:48:59 tokenizer.py:137] Using a slow tokenizer. This might cause a significant slowdown. Consider using a fast tokenizer instead.
INFO 09-03 13:49:04 gpu_executor.py:121] # GPU blocks: 840, # CPU blocks: 728
INFO 09-03 13:49:08 model_runner.py:1181] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 09-03 13:49:08 model_runner.py:1185] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 09-03 13:49:26 model_runner.py:1300] Graph capturing finished in 18 secs.


In [2]:
image = Image.open("21517f0363581d7d_02_09_2024__19_01_05.jpg").convert("RGB")
messages = [{
    "role":
    "user",
    "content":
    # Number of images
    "(<image>./</image>)" + \
    "\nWhich object is shown to the camera? Answer the question using a single word or phrase." 
}]
prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

# Single Inference
inputs = {
    "prompt": prompt,
    "multi_modal_data": {
        "image": image
        # Multi images, the number of images should be equal to that of `(<image>./</image>)`
        # "image": [image, image] 
    },
}
# Batch Inference
# inputs = [{
#     "prompt": prompt,
#     "multi_modal_data": {
#         "image": image
#     },
# } for _ in 2]


# 2.6
stop_tokens = ['<|im_end|>', '<|endoftext|>']
stop_token_ids = [tokenizer.convert_tokens_to_ids(i) for i in stop_tokens]
# 2.0
# stop_token_ids = [tokenizer.eos_id]
# 2.5
# stop_token_ids = [tokenizer.eos_id, tokenizer.eot_id]

sampling_params = SamplingParams(
    stop_token_ids=stop_token_ids, 
    use_beam_search=True,
    temperature=0, 
    best_of=3,
    max_tokens=1024
)

outputs = llm.generate(inputs, sampling_params=sampling_params)

print(outputs[0].outputs[0].text)

ImportError: apply_chat_template requires jinja2>=3.1.0 to be installed. Your version is 3.0.3.

In [10]:
outputs[0].outputs[0].text

'cup'